# Post-Fire Vegetation Recovery: A Decision Support Demonstration

**Purpose**: This notebook demonstrates how simulation modeling can help land managers evaluate post-fire recovery strategies. It is a complete, runnable example of the vegetation modeling component of our post-fire decision support toolbox.

---

## Background: The Post-Fire Decision Problem

After a wildfire, land managers face a critical decision window. They must quickly:

1. **Assess damage** - What burned and how severely?
2. **Predict outcomes** - What will happen if we do nothing? What if we intervene?
3. **Allocate resources** - Where should limited seed, labor, and budget be deployed?

These decisions are complicated by **ecological feedbacks** that can lock landscapes into degraded states:

- Fire kills native trees but temporarily reduces invasive grass cover
- Invasive grasses recover *faster* than native trees
- Once invasives dominate (>50% cover), they block native seedling establishment
- Without intervention, burned areas may flip from native-dominated to invasive-dominated

### Why Simulation?

Traditional approaches to post-fire planning rely on expert judgment or empirical models trained on historical data. But:

- **Experts can't intuit** the outcomes of complex feedbacks over decades
- **Historical data are sparse** for the species and conditions managers care about
- **Novel conditions** (climate change, new invasives) may invalidate past patterns

**Process-based simulation** addresses these limitations by encoding ecological mechanisms (growth rates, mortality, competition) explicitly. The model becomes a "what-if" machine that managers can query: *What happens over 50 years if we plant seedlings here? What if we also remove invasives?*

---

## What This Demonstration Shows

We simulate a **toy ecological system** with:

| Component | Description |
|-----------|-------------|
| **Native Trees** | Individual agents with 3 life stages (seedling, juvenile, adult). Age-dependent mortality, reproduction by adults. |
| **Invasive Grass** | Patch-level cover (0-100%). Grows logistically, suppressed by tree canopy, recovers fast after fire. |
| **Fire** | Spatial severity pattern (0-1). Kills trees based on severity and life stage. Temporarily reduces invasive cover. |
| **Elevation** | Spatial gradient (0-1). High elevation favors native trees; low elevation favors invasives. Creates interpretable patterns. |
| **Interventions** | Seeding (add native seedlings) and invasive removal (reduce grass cover). |

### Elevation as Ecological Driver

Elevation creates a second spatial gradient that makes results interpretable:

| Elevation Zone | Native Tree Survival | Invasive Growth | Management Priority |
|----------------|---------------------|-----------------|---------------------|
| **High** (NW plateau) | Good - reduced seedling mortality | Suppressed | Low - natural recovery likely |
| **Medium** (transition) | Moderate | Moderate | Medium - seeding helps |
| **Low** (SE corner) | Poor - seedlings struggle | Thrives | High - removal critical |

The key insight: *"High elevation burned areas will recover on their own; prioritize management in low elevation burned areas where invasives would otherwise dominate."*

### Scenarios Compared

| Scenario | Fire? | Seeding? | Removal? | Question Answered |
|----------|-------|----------|----------|-------------------|
| `baseline` | No | No | No | What happens without disturbance? (Control) |
| `fire_only` | Yes | No | No | How bad is fire without intervention? |
| `fire_seeding` | Yes | Yes | No | Does planting seedlings help recovery? |
| `fire_removal` | Yes | No | Yes | Does removing invasives help recovery? |
| `fire_both` | Yes | Yes | Yes | Is the combination better than either alone? |

### Workflow Steps

1. **Setup** - Load libraries, configure paths
2. **Generate Spatial Data** - Create fire severity and elevation rasters
3. **Preprocess for Josh** - Convert GeoTIFFs to Josh's optimized format
4. **Validate Model** - Check the simulation model compiles
5. **Run Scenarios** - Execute 5 scenarios (50 years each)
6. **Analyze Results** - Compare outcomes across scenarios

---

## Step 1: Setup and Configuration

**What we're doing**: Loading required Python packages and configuring file paths.

**Why it matters**: Ensures the environment is ready and paths point to the correct files.

In [ ]:
# Standard scientific Python stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Geospatial: for creating GeoTIFF rasters
import rasterio
from rasterio.transform import from_bounds
from rasterio.crs import CRS

# Image processing: for smoothing noise in fire pattern
from scipy.ndimage import uniform_filter

# System utilities
import os
import glob
import subprocess

# Create output directories (organized structure)
os.makedirs('data', exist_ok=True)
os.makedirs('preprocessed', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('analysis/figures/inputs', exist_ok=True)
os.makedirs('analysis/figures/timeseries', exist_ok=True)
os.makedirs('analysis/figures/spatial', exist_ok=True)
os.makedirs('analysis/summaries', exist_ok=True)

print("Environment ready.")

In [ ]:
# =============================================================================
# FILE PATHS
# =============================================================================
# These define where the Josh simulation engine and model files are located.

JOSH_JAR = "jar/joshsim-fat-prod.jar"  # Josh simulation engine (Java)
MODEL_FILE = "vegetation_model.josh"   # The ecological model definition
PARAMS_CONFIG = "configs/params.jshc"  # Shared ecological parameters

# Preprocessed spatial data for Josh
FIRE_DATA = "preprocessed/fire_severity.jshd"
ELEVATION_DATA = "preprocessed/elevation.jshd"

# The five scenarios we'll compare
SCENARIOS = [
    "baseline",      # No fire, no intervention (counterfactual control)
    "fire_only",     # Fire without any management response
    "fire_seeding",  # Fire + plant native seedlings
    "fire_removal",  # Fire + remove invasive grasses
    "fire_both",     # Fire + both interventions combined
]

# Verify the Josh JAR exists
if os.path.exists(JOSH_JAR):
    print(f"Josh simulation engine found: {JOSH_JAR}")
else:
    print(f"ERROR: Josh JAR not found at {JOSH_JAR}")
    print("Please run: bash get_fat_jar.sh")

In [ ]:
# =============================================================================
# SPATIAL CONFIGURATION
# =============================================================================
# These parameters define the simulation grid. They must match the Josh model.

N_ROWS = 30  # Number of patches in Y direction
N_COLS = 30  # Number of patches in X direction

# Geographic bounds in WGS84 degrees
# (Using a small synthetic area; actual location doesn't matter for this demo)
LON_LOW, LON_HIGH = 0.0, 0.003
LAT_LOW, LAT_HIGH = 0.0, 0.003

# Approximate extent in meters (for visualization labels)
X_LOW, X_HIGH = 0, 300
Y_LOW, Y_HIGH = 0, 300

# GeoTIFF transform (maps pixel coordinates to geographic coordinates)
TRANSFORM = from_bounds(LON_LOW, LAT_LOW, LON_HIGH, LAT_HIGH, N_COLS, N_ROWS)

# Common raster profile for GeoTIFF files
BASE_PROFILE = {
    'driver': 'GTiff',
    'width': N_COLS,
    'height': N_ROWS,
    'count': 1,
    'crs': CRS.from_epsg(4326),  # WGS84
    'transform': TRANSFORM,
}

print(f"Simulation grid: {N_ROWS} x {N_COLS} = {N_ROWS * N_COLS} patches")
print(f"Each patch: ~10m x 10m")
print(f"Total area: ~{(X_HIGH-X_LOW)}m x {(Y_HIGH-Y_LOW)}m")

---

## Step 2: Generate Spatial Data

### 2a. Fire Severity

**What we're doing**: Creating a synthetic fire severity raster that simulates a fire spreading from a central ignition point.

**Why it matters**: In a real application, this would come from remote sensing (Sentinel-2, Landsat) or fire behavior models. For this demo, we generate a plausible pattern:

- **High severity (>0.7)** in the center: Most trees killed, soil impacts
- **Moderate severity (0.3-0.7)** in a ring: Some trees killed, patchy survival
- **Low severity (<0.3)** at edges: Light surface fire, most trees survive

The fire pattern drives differential mortality in the simulation and determines where interventions might be most effective.

In [ ]:
def generate_fire_severity(n_rows, n_cols, center, sigma, max_severity, 
                           noise_scale=0.2, seed=None):
    """
    Generate a realistic fire severity pattern.
    
    The pattern combines:
    1. A 2D Gaussian (simulating radial spread from ignition point)
    2. Spatially-correlated noise (terrain/fuel heterogeneity effects)
    3. Wind bias (directional asymmetry in fire spread)
    
    Returns:
        2D array of fire severity values (0-1)
    """
    if seed is not None:
        np.random.seed(seed)
    
    y, x = np.ogrid[:n_rows, :n_cols]
    center_y, center_x = center
    
    # Base pattern: 2D Gaussian centered on ignition point
    distance_sq = (y - center_y)**2 + (x - center_x)**2
    base_severity = max_severity * np.exp(-distance_sq / (2 * sigma**2))
    
    # Add spatially-correlated noise (smoother than random noise)
    raw_noise = np.random.randn(n_rows, n_cols)
    smooth_noise = uniform_filter(raw_noise, size=5, mode='reflect')
    smooth_noise = smooth_noise / (np.abs(smooth_noise).max() + 1e-10)
    
    # Simulate wind effect (fire spreads faster downwind)
    wind_direction = np.random.uniform(0, 2 * np.pi)
    wind_strength = 0.3
    wind_effect = wind_strength * (
        np.cos(wind_direction) * (x - center_x) / n_cols +
        np.sin(wind_direction) * (y - center_y) / n_rows
    )
    
    # Combine: noise is stronger at fire edges (where conditions are marginal)
    edge_factor = 4 * base_severity * (1 - base_severity)
    noise_contribution = noise_scale * smooth_noise * edge_factor
    
    severity = base_severity * (1 + wind_effect) + noise_contribution
    severity = np.clip(severity, 0, 1)
    
    return severity.astype(np.float32)

# Generate the fire severity pattern
FIRE_CENTER = (N_ROWS // 2, N_COLS // 2)  # Ignition at grid center
FIRE_SIGMA = 7   # Controls fire spread (larger = more area burned)
FIRE_MAX = 0.95  # Peak severity at ignition point

fire_severity = generate_fire_severity(
    N_ROWS, N_COLS, FIRE_CENTER, FIRE_SIGMA, FIRE_MAX,
    noise_scale=0.25, seed=42  # Fixed seed for reproducibility
)

# Save as GeoTIFF
fire_profile = BASE_PROFILE.copy()
fire_profile['dtype'] = 'float32'
fire_profile['nodata'] = -9999

with rasterio.open('data/fire_severity.tif', 'w', **fire_profile) as dst:
    dst.write(fire_severity, 1)

# Summary statistics
print("Fire Severity Summary:")
print(f"  Range: {fire_severity.min():.2f} to {fire_severity.max():.2f}")
print(f"  High severity (>0.7): {(fire_severity > 0.7).sum()} patches ({100*(fire_severity > 0.7).mean():.1f}%)")
print(f"  Moderate (0.3-0.7): {((fire_severity > 0.3) & (fire_severity <= 0.7)).sum()} patches")
print(f"  Low (<0.3): {(fire_severity <= 0.3).sum()} patches")
print(f"\nSaved: data/fire_severity.tif")

In [ ]:
# Visualize the fire severity pattern
fig, ax = plt.subplots(figsize=(8, 8))

im = ax.imshow(fire_severity, cmap='YlOrRd', vmin=0, vmax=1, origin='lower',
               extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
ax.set_title('Fire Severity Pattern', fontsize=14)
ax.set_xlabel('X position (meters)')
ax.set_ylabel('Y position (meters)')

# Show severity threshold contours
contours = ax.contour(
    np.linspace(X_LOW, X_HIGH, N_COLS), 
    np.linspace(Y_LOW, Y_HIGH, N_ROWS),
    fire_severity, 
    levels=[0.3, 0.7], 
    colors=['orange', 'red'], 
    linestyles=['--', '-'], 
    linewidths=2
)
ax.clabel(contours, inline=True, fontsize=10, fmt='%.1f')

cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Severity (0=unburned, 1=complete consumption)', fontsize=11)

plt.tight_layout()
plt.savefig('analysis/figures/inputs/fire_severity_preview.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nSeverity thresholds used in the model:")
print("  - 0.3 (orange dashed): Below this, low fire mortality")
print("  - 0.7 (red solid): Above this, high fire mortality")

### 2b. Elevation Gradient

**What we're doing**: Creating a synthetic elevation raster with a NW-SE gradient plus a high-elevation plateau.

**Why it matters**: Elevation creates spatial structure in ecological outcomes that makes difference maps interpretable:

- **Low elevation (SE)**: Invasive grasses thrive (warmer, drier conditions favor them). Native seedlings struggle with drought stress.
- **High elevation (NW plateau)**: Native trees recover better (cooler, moister). Invasives are climate-limited.
- **Transition zone**: Where fire + management interactions are most visible.

**Key parameters in `configs/params.jshc`:**
- `invasiveElevationSensitivity`: How much elevation suppresses invasive growth (0.0-1.0)
- `seedlingElevationBonus`: How much elevation helps seedling survival (0.0-1.0)
- `juvenileElevationBonus`: How much elevation helps juvenile survival (0.0-1.0)

**Tuning guide**: If spatial patterns are too subtle, increase these values. If invasives never grow at high elevation, decrease sensitivity.

In [ ]:
def generate_elevation(n_rows, n_cols, plateau_fraction=0.4, plateau_boost=0.3, seed=None):
    """
    Generate an elevation raster with a NW-SE gradient plus a high-elevation plateau.
    
    The pattern creates clear ecological zones:
    - NW corner + plateau: High elevation (trees thrive, invasives suppressed)
    - SE corner: Low elevation (invasives thrive, seedlings struggle)
    - Center: Transition zone (where fire + management interactions are visible)
    
    Parameters:
        n_rows, n_cols: Grid dimensions
        plateau_fraction: Fraction of grid covered by plateau (0.0-1.0)
        plateau_boost: Additional elevation for plateau area (0.0-1.0)
        seed: Random seed for reproducibility
        
    Returns:
        2D array of elevation values (0-100 percent scale)
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Create coordinate grids (0-1 normalized)
    y, x = np.ogrid[:n_rows, :n_cols]
    y_norm = y / (n_rows - 1)
    x_norm = x / (n_cols - 1)
    
    # Base gradient: NW high, SE low
    # y=0 is bottom (south), y=max is top (north)
    # x=0 is left (west), x=max is right (east)
    # We want NW (top-left) to be high elevation
    gradient = (1 - x_norm) * 0.5 + y_norm * 0.5
    
    # Add plateau in upper-left portion (sudden step up)
    # This creates a "refugium" at high elevation
    plateau_mask = (y_norm > (1 - plateau_fraction)) & (x_norm < plateau_fraction)
    elevation = gradient + plateau_mask * plateau_boost
    
    # Add subtle noise for realism (but keep patterns clear)
    noise = np.random.randn(n_rows, n_cols) * 0.03
    elevation = elevation + noise
    
    # Normalize to 0-1 range, then scale to 0-100 (percent)
    elevation = (elevation - elevation.min()) / (elevation.max() - elevation.min())
    elevation = elevation * 100  # Convert to 0-100 percent scale
    
    return elevation.astype(np.float32)

# =============================================================================
# ELEVATION PARAMETERS (adjust these to tune ecological patterns)
# =============================================================================
# plateau_fraction: How much of the NW corner is plateau (0.4 = 40%)
# plateau_boost: How much higher the plateau is (0.3 = 30% elevation boost)
# These interact with params.jshc settings to determine ecological outcomes

PLATEAU_FRACTION = 0.4  # Fraction of grid that's high-elevation plateau
PLATEAU_BOOST = 0.3     # Additional elevation for plateau area

elevation = generate_elevation(
    N_ROWS, N_COLS, 
    plateau_fraction=PLATEAU_FRACTION,
    plateau_boost=PLATEAU_BOOST,
    seed=42  # Fixed seed for reproducibility
)

# Save as GeoTIFF
elevation_profile = BASE_PROFILE.copy()
elevation_profile['dtype'] = 'float32'
elevation_profile['nodata'] = -9999

with rasterio.open('data/elevation.tif', 'w', **elevation_profile) as dst:
    dst.write(elevation, 1)

# Summary statistics (now in percent 0-100)
print("Elevation Summary (0-100% scale):")
print(f"  Range: {elevation.min():.1f}% to {elevation.max():.1f}%")
print(f"  High elevation (>60%): {(elevation > 60).sum()} patches ({100*(elevation > 60).mean():.1f}%)")
print(f"  Medium (30-60%): {((elevation > 30) & (elevation <= 60)).sum()} patches")
print(f"  Low (<30%): {(elevation <= 30).sum()} patches")
print(f"\nSaved: data/elevation.tif")

In [ ]:
# Visualize elevation pattern
fig, ax = plt.subplots(figsize=(8, 8))

im = ax.imshow(elevation, cmap='terrain', vmin=0, vmax=100, origin='lower',
               extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
ax.set_title('Elevation Gradient (0-100%)', fontsize=14)
ax.set_xlabel('X position (meters)')
ax.set_ylabel('Y position (meters)')

# Show elevation zone contours
contours = ax.contour(
    np.linspace(X_LOW, X_HIGH, N_COLS), 
    np.linspace(Y_LOW, Y_HIGH, N_ROWS),
    elevation, 
    levels=[30, 60], 
    colors=['brown', 'darkgreen'], 
    linestyles=['--', '-'], 
    linewidths=1.5
)
ax.clabel(contours, inline=True, fontsize=10, fmt='%.0f%%')

cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Elevation (0%=low, 100%=high)', fontsize=11)

# Add annotations for ecological zones
ax.annotate('HIGH ELEV\n(>60%)', xy=(50, 250), fontsize=10, 
            color='white', fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='darkgreen', alpha=0.7))
ax.annotate('LOW ELEV\n(<30%)', xy=(250, 50), fontsize=10, 
            color='white', fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='brown', alpha=0.7))

plt.tight_layout()
plt.savefig('analysis/figures/inputs/elevation_preview.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nElevation zones and ecological effects:")
print("  - High (>60%): Seedling survival bonus, invasive growth suppressed")
print("  - Medium (30-60%): Intermediate conditions (partial bonus)")
print("  - Low (<30%): Seedlings struggle, invasives thrive")

In [ ]:
# Combined view: Fire severity + Elevation
# This shows how the two gradients interact to create management priority zones

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Panel 1: Fire severity
ax = axes[0]
im = ax.imshow(fire_severity, cmap='YlOrRd', vmin=0, vmax=1, origin='lower',
               extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
ax.contour(np.linspace(X_LOW, X_HIGH, N_COLS), np.linspace(Y_LOW, Y_HIGH, N_ROWS),
           fire_severity, levels=[0.3, 0.7], colors=['orange', 'red'], linewidths=1.5)
ax.set_title('Fire Severity', fontsize=12, fontweight='bold')
ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
plt.colorbar(im, ax=ax, shrink=0.8, label='Severity')

# Panel 2: Elevation (0-100% scale)
ax = axes[1]
im = ax.imshow(elevation, cmap='terrain', vmin=0, vmax=100, origin='lower',
               extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
ax.contour(np.linspace(X_LOW, X_HIGH, N_COLS), np.linspace(Y_LOW, Y_HIGH, N_ROWS),
           elevation, levels=[30, 60], colors=['brown', 'green'], linewidths=1.5)
ax.set_title('Elevation', fontsize=12, fontweight='bold')
ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
plt.colorbar(im, ax=ax, shrink=0.8, label='Elevation (%)')

# Panel 3: Management Priority (fire severity * inverse elevation)
# High priority = high severity AND low elevation
# This is where invasives will take over without intervention
# Scale elevation to 0-1 for calculation
elevation_01 = elevation / 100
management_priority = fire_severity * (1 - elevation_01)
ax = axes[2]
im = ax.imshow(management_priority, cmap='RdYlBu_r', vmin=0, vmax=1, origin='lower',
               extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
ax.contour(np.linspace(X_LOW, X_HIGH, N_COLS), np.linspace(Y_LOW, Y_HIGH, N_ROWS),
           fire_severity, levels=[0.7], colors=['black'], linestyles=['--'], linewidths=1.5)
ax.contour(np.linspace(X_LOW, X_HIGH, N_COLS), np.linspace(Y_LOW, Y_HIGH, N_ROWS),
           elevation, levels=[60], colors=['white'], linestyles=[':'], linewidths=1.5)
ax.set_title('Management Priority\n(high severity + low elevation)', fontsize=12, fontweight='bold')
ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
plt.colorbar(im, ax=ax, shrink=0.8, label='Priority')

plt.tight_layout()
plt.savefig('analysis/figures/inputs/all_inputs_preview.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nManagement priority interpretation:")
print("  - RED zones: High fire severity + low elevation = URGENT intervention needed")
print("  - BLUE zones: Low priority (unburned or high elevation = natural recovery likely)")
print("  - Dashed line: High severity fire boundary (0.7)")
print("  - Dotted line: High elevation boundary (60%)")

---

## Step 3: Preprocess Data for Josh

**What we're doing**: Converting GeoTIFF rasters (fire severity, elevation) into Josh's optimized binary format (`.jshd`).

**Why it matters**: Josh reads spatial data from `.jshd` files during simulation. The preprocessing step:

1. Aligns the raster grid to the simulation grid
2. Handles coordinate system transformations
3. Creates an indexed format for fast random access

This is analogous to how GIS software preprocesses rasters for analysis.

In [ ]:
# Preprocess fire severity and elevation rasters for Josh
print("=" * 60)
print("PREPROCESSING SPATIAL DATA FOR JOSH")
print("=" * 60)

# Preprocess fire severity
print("\n=== Preprocessing fire severity ===")
print(f"Input:  data/fire_severity.tif")
print(f"Output: preprocessed/fire_severity.jshd")

result = subprocess.run(
    [
        'java', '-jar', JOSH_JAR, 'preprocess',
        MODEL_FILE, 'Main',
        'data/fire_severity.tif', '0', 'ratio',
        'preprocessed/fire_severity.jshd',
        '--timestep', '0'
    ],
    capture_output=True, text=True
)

if result.returncode == 0:
    size = os.path.getsize('preprocessed/fire_severity.jshd')
    print(f"Complete. Output size: {size/1024:.1f} KB")
else:
    print(f"ERROR: {result.stderr}")

# Preprocess elevation
print("\n=== Preprocessing elevation ===")
print(f"Input:  data/elevation.tif")
print(f"Output: preprocessed/elevation.jshd")

result = subprocess.run(
    [
        'java', '-jar', JOSH_JAR, 'preprocess',
        MODEL_FILE, 'Main',
        'data/elevation.tif', '0', 'ratio',
        'preprocessed/elevation.jshd',
        '--timestep', '0'
    ],
    capture_output=True, text=True
)

if result.returncode == 0:
    size = os.path.getsize('preprocessed/elevation.jshd')
    print(f"Complete. Output size: {size/1024:.1f} KB")
else:
    print(f"ERROR: {result.stderr}")

print("\n" + "=" * 60)
print("PREPROCESSING COMPLETE")
print("=" * 60)

---

## Step 4: Validate Model and Review Configuration

**What we're doing**: Checking that the Josh model compiles correctly and reviewing the configuration files.

**Why it matters**: The model defines all the ecological mechanisms (mortality rates, competition, reproduction). Configuration files let us run different scenarios without changing the model code.

### Model Architecture

The simulation uses **separation of concerns**:

| File | Contains | Changes when... |
|------|----------|----------------|
| `vegetation_model.josh` | Ecological mechanisms | Scientific understanding improves |
| `configs/params.jshc` | Parameter values | Calibration/tuning |
| `configs/<scenario>.jshc` | Scenario settings | Running new experiments |

In [ ]:
# Validate the Josh model
print("=== Validating Josh Model ===")

result = subprocess.run(
    ['java', '-jar', JOSH_JAR, 'validate', MODEL_FILE],
    capture_output=True, text=True
)

# Filter out the pixi warning
output = result.stdout + result.stderr
for line in output.split('\n'):
    if 'Validated' in line or 'Error' in line:
        print(line)

if result.returncode == 0:
    print("\nModel validation: PASSED")
else:
    print("\nModel validation: FAILED")
    print(result.stderr)

In [ ]:
# Display the scenario configuration files
print("=" * 60)
print("SCENARIO CONFIGURATIONS")
print("=" * 60)
print("\nThese files control what happens in each scenario.")
print("Key variables:")
print("  - hasFire: Whether fire occurred (0=no, 1=yes)")
print("  - seedingBoost: Extra seedlings to plant (count)")
print("  - removalEffort: Invasive removal rate (percent/year)")

for scenario in SCENARIOS:
    print(f"\n{'='*40}")
    print(f"Scenario: {scenario}")
    print(f"{'='*40}")
    with open(f'configs/{scenario}.jshc', 'r') as f:
        print(f.read())

---

## Step 5: Run Simulation Scenarios

**What we're doing**: Running 5 simulation scenarios, each projecting 50 years of vegetation dynamics.

**Why it matters**: By comparing scenarios, managers can answer key questions:

- **Fire impact**: How much worse is the fire scenario vs. baseline?
- **Seeding value**: Does planting seedlings improve recovery?
- **Removal value**: Does removing invasives improve recovery?
- **Synergy**: Is doing both better than either alone?

With **elevation as a spatial driver**, we expect clearer patterns in the difference maps:
- High elevation burned areas should recover naturally (less intervention needed)
- Low elevation burned areas should show the biggest intervention benefits

### How Josh's Config System Works

Each scenario is run with the same model but different configuration:

```bash
java -jar joshsim.jar run \
    --data params.jshc=configs/params.jshc \      # Shared parameters
    --data scenario.jshc=configs/baseline.jshc \  # Scenario-specific settings
    --data fire.jshd=preprocessed/fire_severity.jshd \  # Fire severity map
    --data elevation.jshd=preprocessed/elevation.jshd \ # Elevation map
    vegetation_model.josh Main
```

This design means:
- One model file serves all scenarios
- Adding new scenarios requires only a new `.jshc` file
- Configuration is transparent and auditable

In [ ]:
# Run all scenarios
print("="*60)
print("RUNNING SIMULATION SCENARIOS")
print("="*60)
print(f"\nModel: {MODEL_FILE}")
print(f"Parameters: {PARAMS_CONFIG}")
print(f"Fire data: {FIRE_DATA}")
print(f"Elevation data: {ELEVATION_DATA}")
print(f"\nScenarios: {', '.join(SCENARIOS)}")
print()

for scenario in SCENARIOS:
    print(f"--- Running: {scenario} ---")
    
    # Run Josh simulation with both fire and elevation data
    result = subprocess.run(
        [
            'java', '-jar', JOSH_JAR, 'run',
            '--data', f'params.jshc={PARAMS_CONFIG}',
            '--data', f'scenario.jshc=configs/{scenario}.jshc',
            '--data', f'fire.jshd={FIRE_DATA}',
            '--data', f'elevation.jshd={ELEVATION_DATA}',
            MODEL_FILE, 'Main'
        ],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        # Rename output file to include scenario name
        if os.path.exists('results/output_0.csv'):
            os.rename('results/output_0.csv', f'results/{scenario}_0.csv')
            size = os.path.getsize(f'results/{scenario}_0.csv') / (1024*1024)
            print(f"    Completed. Output: results/{scenario}_0.csv ({size:.1f} MB)")
        else:
            print(f"    Warning: No output file generated")
    else:
        print(f"    ERROR: {result.stderr[:200]}")

print()
print("="*60)
print("ALL SCENARIOS COMPLETE")
print("="*60)

---

## Step 6: Load and Analyze Results

**What we're doing**: Loading simulation outputs and computing summary statistics.

**Why it matters**: Raw simulation output is patch-level data for each time step. We aggregate this to answer landscape-scale questions:

- Total tree population over time
- Mean invasive cover over time
- Recovery trajectories under different scenarios

In [ ]:
# Load results from all scenarios
print("Loading simulation results...\n")

all_data = []

for scenario in SCENARIOS:
    pattern = f'results/{scenario}_*.csv'
    files = sorted(glob.glob(pattern))
    
    if not files:
        print(f"  WARNING: No results found for '{scenario}'")
        continue
    
    for i, filepath in enumerate(files):
        df = pd.read_csv(filepath)
        df['scenario'] = scenario
        df['replicate'] = i
        all_data.append(df)
    
    print(f"  {scenario}: {len(files)} file(s), {len(df):,} rows each")

if all_data:
    data = pd.concat(all_data, ignore_index=True)
    print(f"\nTotal: {len(data):,} rows loaded")
    print(f"Columns: {', '.join(data.columns[:8])}...")
else:
    print("\nERROR: No data loaded. Run the simulation step above first.")
    data = None

In [ ]:
# Aggregate data for time series analysis
if data is not None:
    # Sum trees and average invasive cover across all patches
    time_series = data.groupby(['scenario', 'replicate', 'step']).agg({
        'numSeedling': 'sum',
        'numJuvenile': 'sum',
        'numAdult': 'sum',
        'numAlive': 'sum',
        'invasiveCover': 'mean',
        'newSeedlings': 'sum',
    }).reset_index()
    
    # Calculate summary statistics (mean +/- std across replicates)
    summary = time_series.groupby(['scenario', 'step']).agg({
        'numAlive': ['mean', 'std'],
        'invasiveCover': ['mean', 'std'],
    }).reset_index()
    summary.columns = ['scenario', 'step', 'trees_mean', 'trees_std', 
                       'invasive_mean', 'invasive_std']
    
    # Fill NaN std (happens with single replicate)
    summary['trees_std'] = summary['trees_std'].fillna(0)
    summary['invasive_std'] = summary['invasive_std'].fillna(0)
    
    print("Aggregation complete.")
    print(f"Time steps: {time_series['step'].min()} to {time_series['step'].max()} (years)")
    print(f"Total patches per scenario: {N_ROWS * N_COLS}")

---

## Step 7: Visualize Results

**What we're doing**: Creating plots that compare outcomes across scenarios.

**Why it matters**: Visualization makes complex simulation output interpretable. Land managers can see:

- **Trajectories**: How populations change over 50 years
- **Endpoint comparisons**: Final state under each scenario
- **Intervention effectiveness**: Which strategies produce better outcomes

In [ ]:
# Time series comparison: Tree population and invasive cover
if data is not None:
    # Color scheme for scenarios
    colors = {
        'baseline': '#2ecc71',      # Green - reference
        'fire_only': '#e74c3c',     # Red - worst case
        'fire_seeding': '#3498db',  # Blue - seeding intervention
        'fire_removal': '#9b59b6',  # Purple - removal intervention
        'fire_both': '#f39c12',     # Orange - combined
    }
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left panel: Tree population over time
    ax1 = axes[0]
    for scenario in SCENARIOS:
        subset = summary[summary['scenario'] == scenario]
        ax1.plot(subset['step'], subset['trees_mean'], 
                 color=colors[scenario], label=scenario, linewidth=2)
        # Add uncertainty band if multiple replicates
        if subset['trees_std'].max() > 0:
            ax1.fill_between(subset['step'], 
                             subset['trees_mean'] - subset['trees_std'],
                             subset['trees_mean'] + subset['trees_std'],
                             color=colors[scenario], alpha=0.2)
    
    ax1.set_xlabel('Year', fontsize=12)
    ax1.set_ylabel('Total Living Trees', fontsize=12)
    ax1.set_title('Tree Population Over Time', fontsize=14)
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 50)
    
    # Right panel: Invasive cover over time
    ax2 = axes[1]
    for scenario in SCENARIOS:
        subset = summary[summary['scenario'] == scenario]
        ax2.plot(subset['step'], subset['invasive_mean'], 
                 color=colors[scenario], label=scenario, linewidth=2)
        if subset['invasive_std'].max() > 0:
            ax2.fill_between(subset['step'], 
                             subset['invasive_mean'] - subset['invasive_std'],
                             subset['invasive_mean'] + subset['invasive_std'],
                             color=colors[scenario], alpha=0.2)
    
    ax2.set_xlabel('Year', fontsize=12)
    ax2.set_ylabel('Mean Invasive Cover (%)', fontsize=12)
    ax2.set_title('Invasive Grass Cover Over Time', fontsize=14)
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, 50)
    
    plt.tight_layout()
    plt.savefig('analysis/figures/timeseries/scenario_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nInterpretation:")
    print("- Baseline (green): No fire, stable reference condition")
    print("- Fire only (red): Fire without intervention - worst recovery")
    print("- Interventions (blue/purple/orange): Do they improve on fire-only?")

In [ ]:
# Final state summary (Year 50)
if data is not None:
    max_step = data['step'].max()
    final = time_series[time_series['step'] == max_step].copy()
    
    print(f"\n{'='*60}")
    print(f"FINAL STATE SUMMARY (Year {max_step})")
    print(f"{'='*60}")
    
    print("\nTotal Trees (landscape-wide):")
    baseline_trees = final[final['scenario'] == 'baseline']['numAlive'].mean()
    for scenario in SCENARIOS:
        if scenario in final['scenario'].values:
            mean = final[final['scenario'] == scenario]['numAlive'].mean()
            pct_of_baseline = 100 * mean / baseline_trees if baseline_trees > 0 else 0
            print(f"  {scenario:15s}: {mean:8,.0f} trees ({pct_of_baseline:5.1f}% of baseline)")
    
    print("\nMean Invasive Cover:")
    for scenario in SCENARIOS:
        if scenario in final['scenario'].values:
            mean = final[final['scenario'] == scenario]['invasiveCover'].mean()
            print(f"  {scenario:15s}: {mean:5.1f}%")

In [ ]:
# Bar chart comparison of final states
if data is not None:
    max_step = data['step'].max()
    final = time_series[time_series['step'] == max_step]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Get data for available scenarios
    available = [s for s in SCENARIOS if s in final['scenario'].values]
    tree_means = [final[final['scenario'] == s]['numAlive'].mean() for s in available]
    invasive_means = [final[final['scenario'] == s]['invasiveCover'].mean() for s in available]
    
    x = np.arange(len(available))
    bar_colors = [colors[s] for s in available]
    
    # Tree population
    axes[0].bar(x, tree_means, color=bar_colors, alpha=0.8)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(available, rotation=45, ha='right')
    axes[0].set_ylabel('Total Trees')
    axes[0].set_title(f'Tree Population at Year {max_step}')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Invasive cover
    axes[1].bar(x, invasive_means, color=bar_colors, alpha=0.8)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(available, rotation=45, ha='right')
    axes[1].set_ylabel('Mean Invasive Cover (%)')
    axes[1].set_title(f'Invasive Cover at Year {max_step}')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('analysis/figures/timeseries/final_state_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

---

### Spatial Comparison: Where Do Interventions Help?

**What we're doing**: Creating maps that show the *difference* between scenarios at each location.

**Why it matters**: Aggregate statistics hide spatial patterns. Managers need to know:
- **Where** fire caused the most damage
- **Where** interventions provide the most benefit
- **Where** recovery is still lagging despite intervention

**With elevation as a driver**, we expect:
- Fire impact concentrated in **low elevation** burned areas (where seedlings struggle)
- Seeding benefit visible in **moderate elevation** burned zones
- Removal benefit strongest in **low elevation** burned areas (where invasives thrive)
- Combined benefit showing clear "priority zones" in low-moderate elevation fire areas

Color scale:
- **Green** = Better than reference (more trees or intervention helped)
- **Red** = Worse than reference (fewer trees or fire damage persists)
- **White** = No difference

Contour lines:
- **Black dashed**: High fire severity boundary (0.7)
- **White dotted**: Medium elevation contour (0.5)

In [ ]:
# Spatial difference maps: comparing scenarios at final time step
if data is not None:
    from matplotlib.colors import TwoSlopeNorm
    
    max_step = data['step'].max()
    final_data = data[data['step'] == max_step].copy()
    
    # Determine which position columns to use
    if 'position.x' in final_data.columns:
        x_col, y_col = 'position.x', 'position.y'
    else:
        x_col, y_col = 'x', 'y'
    
    # Function to create spatial grid from scenario data
    def scenario_to_grid(scenario_name, variable='numAlive'):
        subset = final_data[final_data['scenario'] == scenario_name]
        grid = np.zeros((N_ROWS, N_COLS))
        for _, row in subset.iterrows():
            x_idx = int(row[x_col])
            y_idx = int(row[y_col])
            if 0 <= x_idx < N_COLS and 0 <= y_idx < N_ROWS:
                grid[y_idx, x_idx] = row[variable]
        return grid
    
    # Get grids for each scenario
    baseline_trees = scenario_to_grid('baseline', 'numAlive')
    fire_only_trees = scenario_to_grid('fire_only', 'numAlive')
    fire_seeding_trees = scenario_to_grid('fire_seeding', 'numAlive')
    fire_removal_trees = scenario_to_grid('fire_removal', 'numAlive')
    fire_both_trees = scenario_to_grid('fire_both', 'numAlive')
    
    # Calculate differences
    # Negative = worse than reference, Positive = better than reference
    fire_impact = fire_only_trees - baseline_trees  # Negative where fire hurt
    seeding_benefit = fire_seeding_trees - fire_only_trees  # Positive where seeding helped
    removal_benefit = fire_removal_trees - fire_only_trees  # Positive where removal helped
    combined_benefit = fire_both_trees - fire_only_trees  # Positive where combined helped
    
    # Find max absolute value for symmetric colorbar
    all_diffs = [fire_impact, seeding_benefit, removal_benefit, combined_benefit]
    vmax = max(np.abs(d).max() for d in all_diffs)
    vmax = max(vmax, 1)  # Ensure at least 1 for colorbar
    
    # Create figure with 2x2 subplots
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    # Diverging colormap: red = worse, white = neutral, green = better
    cmap = plt.cm.RdYlGn
    
    # Coordinate arrays for contours
    x_coords = np.linspace(X_LOW, X_HIGH, N_COLS)
    y_coords = np.linspace(Y_LOW, Y_HIGH, N_ROWS)
    
    def add_contours(ax):
        """Add fire severity and elevation contours to a plot."""
        # Fire severity contour (high severity boundary)
        ax.contour(x_coords, y_coords, fire_severity, levels=[0.7], 
                   colors=['black'], linestyles=['--'], linewidths=1.5, alpha=0.7)
        # Elevation contour (high elevation boundary, using 60% threshold)
        ax.contour(x_coords, y_coords, elevation, levels=[60], 
                   colors=['white'], linestyles=[':'], linewidths=2, alpha=0.9)
    
    # Plot 1: Fire Impact (fire_only - baseline)
    ax = axes[0, 0]
    norm = TwoSlopeNorm(vmin=-vmax, vcenter=0, vmax=vmax)
    im = ax.imshow(fire_impact, cmap=cmap, norm=norm, origin='lower',
                   extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
    ax.set_title('Fire Impact\n(fire_only - baseline)', fontsize=12, fontweight='bold')
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Tree difference', fontsize=10)
    add_contours(ax)
    
    # Plot 2: Seeding Benefit (fire_seeding - fire_only)
    ax = axes[0, 1]
    im = ax.imshow(seeding_benefit, cmap=cmap, norm=norm, origin='lower',
                   extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
    ax.set_title('Seeding Benefit\n(fire_seeding - fire_only)', fontsize=12, fontweight='bold')
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Tree difference', fontsize=10)
    add_contours(ax)
    
    # Plot 3: Removal Benefit (fire_removal - fire_only)
    ax = axes[1, 0]
    im = ax.imshow(removal_benefit, cmap=cmap, norm=norm, origin='lower',
                   extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
    ax.set_title('Invasive Removal Benefit\n(fire_removal - fire_only)', fontsize=12, fontweight='bold')
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Tree difference', fontsize=10)
    add_contours(ax)
    
    # Plot 4: Combined Benefit (fire_both - fire_only)
    ax = axes[1, 1]
    im = ax.imshow(combined_benefit, cmap=cmap, norm=norm, origin='lower',
                   extent=[X_LOW, X_HIGH, Y_LOW, Y_HIGH])
    ax.set_title('Combined Intervention Benefit\n(fire_both - fire_only)', fontsize=12, fontweight='bold')
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Tree difference', fontsize=10)
    add_contours(ax)
    
    plt.suptitle(f'Spatial Comparison at Year {max_step}\n'
                 f'Green = more trees, Red = fewer trees\n'
                 f'Black dashed = high fire severity (0.7), White dotted = high elevation (60%)',
                 fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig('analysis/figures/spatial/spatial_difference_maps.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print summary statistics
    print("\nSpatial Summary:")
    print(f"  Fire Impact: {fire_impact.sum():.0f} total trees lost across landscape")
    print(f"  Seeding Benefit: {seeding_benefit.sum():.0f} additional trees from seeding")
    print(f"  Removal Benefit: {removal_benefit.sum():.0f} additional trees from removal")
    print(f"  Combined Benefit: {combined_benefit.sum():.0f} additional trees from both")
    
    # Calculate correlation with elevation (scale to 0-1 for correlation)
    elev_flat = (elevation / 100).flatten()
    print("\nElevation correlation (expected pattern: interventions help more at low elevation):")
    for name, diff in [('Fire Impact', fire_impact), ('Seeding Benefit', seeding_benefit), 
                       ('Removal Benefit', removal_benefit), ('Combined Benefit', combined_benefit)]:
        corr = np.corrcoef(elev_flat, diff.flatten())[0, 1]
        print(f"  {name:20s}: r = {corr:+.2f}")

In [ ]:
# Export combined results
if data is not None:
    data.to_csv('analysis/summaries/all_scenarios_combined.csv', index=False)
    print(f"Exported: analysis/summaries/all_scenarios_combined.csv ({len(data):,} rows)")
    
    summary.to_csv('analysis/summaries/time_series_summary.csv', index=False)
    print(f"Exported: analysis/summaries/time_series_summary.csv")

---

## Step 8: Publication-Quality R Visualizations

**What we're doing**: Running an R script that generates publication-quality figures using ggplot2.

**Why it matters**: While Python's matplotlib is great for exploratory analysis, R's ggplot2 produces cleaner, more polished figures suitable for publications and presentations. The R script creates 9 comprehensive figures:

| Figure | Description | Location |
|--------|-------------|----------|
| fig1 | Tree population time series | `timeseries/` |
| fig2 | Invasive cover time series | `timeseries/` |
| fig3 | Final tree population bars | `comparisons/` |
| fig4 | Fire severity spatial map | `spatial/` |
| fig5 | Intervention effectiveness | `comparisons/` |
| fig6 | Life stage composition | `timeseries/` |
| fig7 | Scenario dashboard | `comparisons/` |
| fig8 | All metrics by scenario | `comparisons/` |
| fig9 | Intervention comparison | `comparisons/` |

These figures complement the Python visualizations above with additional analytical views.

In [ ]:
# Run R visualization script
print("="*60)
print("GENERATING R VISUALIZATIONS")
print("="*60)
print("\nRunning: analysis/visualizations.R")
print("This creates 9 publication-quality figures using ggplot2.\n")

# Create the comparisons directory
os.makedirs('analysis/figures/comparisons', exist_ok=True)

# Run R script
result = subprocess.run(
    ['Rscript', 'analysis/visualizations.R'],
    capture_output=True, text=True, cwd='/workspace'
)

# Display output
if result.stdout:
    print(result.stdout)
if result.stderr:
    # Filter out common R warnings that aren't errors
    stderr_lines = [l for l in result.stderr.split('\n') 
                    if l and not l.startswith('Warning') and 'Joining' not in l]
    if stderr_lines:
        print("Warnings/Errors:")
        for line in stderr_lines[:10]:  # Limit output
            print(f"  {line}")

if result.returncode == 0:
    print("\n" + "="*60)
    print("R VISUALIZATIONS COMPLETE")
    print("="*60)
    print("\nGenerated figures:")
    for subdir in ['timeseries', 'spatial', 'comparisons']:
        path = f'analysis/figures/{subdir}'
        if os.path.exists(path):
            files = [f for f in os.listdir(path) if f.endswith('.png')]
            if files:
                print(f"\n  {subdir}/")
                for f in sorted(files):
                    print(f"    - {f}")
else:
    print(f"\nERROR: R script failed with return code {result.returncode}")

### Figure 1: Tree Population Recovery

- **Baseline (blue)**: Stable equilibrium without fire
- **Fire only (red)**: Slowest recovery without intervention  
- **Interventions**: All improve recovery trajectories

![Tree Population](analysis/figures/timeseries/fig1_tree_population_comparison.png)

### Figure 2: Invasive Cover Dynamics

- **Dashed line at 50%**: Establishment threshold - above this, native seedlings cannot establish
- **Fire only (red)**: Invasives spike and block native recovery
- **Removal scenarios**: Keep cover below threshold, enabling recovery

![Invasive Cover](analysis/figures/timeseries/fig2_invasive_cover_comparison.png)

### Figure 5: Intervention Effectiveness

- **0%** = Fire-only outcome (no recovery)
- **100%** = Full recovery to baseline
- Combined interventions achieve best results; removal more effective than seeding alone

![Intervention Effectiveness](analysis/figures/comparisons/fig5_intervention_effectiveness.png)

### Figure 6: Life Stage Composition

- **Seedlings** (green): Age 0-3, highest mortality
- **Juveniles** (orange): Age 3-10, moderate mortality  
- **Adults** (purple): Age 10+, produce new seedlings

![Life Stages](analysis/figures/timeseries/fig6_life_stages_by_scenario.png)

### Figure 9: Fire Scenario Comparison

Top panel: Tree population diverges as interventions take effect.
Bottom panel: Scenarios with removal stay below 50% threshold; those without exceed it.

![Fire Scenarios](analysis/figures/comparisons/fig9_intervention_comparison.png)

### Figure 3: Final Tree Population

Dashed line = baseline (no fire). Error bars = standard deviation.

![Final Trees](analysis/figures/comparisons/fig3_final_tree_population.png)

### Figure 7: Scenario Comparison Dashboard

**Multi-panel overview** showing both metrics (trees and invasive cover) for all scenarios:

- **Top row**: Total tree population over time
- **Bottom row**: Invasive cover (%) over time
- **Columns**: Each scenario from baseline to combined intervention

**Use this figure to:**
- Compare trajectories at a glance across all scenarios
- Identify when scenarios diverge from each other
- See the relationship between invasive cover and tree recovery

![Scenario Dashboard](analysis/figures/comparisons/fig7_scenario_dashboard.png)

---

## Analysis Summary

### Key Findings from the Simulation

1. **Fire creates a recovery deficit**: Even after 50 years, fire-affected areas have fewer trees than baseline (Figure 1, 3)

2. **Invasive grasses are the primary barrier to recovery**: The 50% establishment threshold creates a tipping point that can lock the system in a degraded state (Figure 2)

3. **Removal is more effective than seeding alone**: Addressing the root cause (invasive competition) is more impactful than simply adding more seedlings (Figure 5)

4. **Combined interventions show synergy**: Doing both seeding AND removal produces better outcomes than either alone (Figures 5, 9)

5. **Timing matters**: The demographic bottleneck occurs in the seedling stage, so interventions that protect seedlings are most valuable (Figure 6)

### Management Recommendations

Based on this simulation:

| Priority | Action | Rationale |
|----------|--------|-----------|
| **High** | Invasive removal in burned areas | Keeps cover below establishment threshold |
| **Medium** | Seeding in treated areas | Accelerates recovery once invasives controlled |
| **Lower** | Seeding without removal | Limited benefit if invasives dominate |

### Caveats

This is a **demonstration model** with simplified dynamics. Real-world applications should:
- Calibrate parameters to local species and conditions
- Include spatial seed dispersal between patches
- Account for climate variability and drought stress
- Validate against monitoring data from actual post-fire sites

---

## Spatial Analysis: Where Should We Intervene?

**This is the key output for land managers.** The spatial difference maps below answer the critical question: *Where on the landscape do interventions provide the most benefit?*

These maps show the **difference in tree counts** between scenarios at each location:
- **Green** = More trees (intervention helped)
- **Red** = Fewer trees (fire damage or intervention didn't help)
- **White** = No difference

**Overlay contours:**
- **Black dashed line**: High fire severity boundary (0.7) - areas of most intense burn
- **White dotted line**: High elevation boundary (60%) - areas where natural recovery is expected

**The management insight**: Focus resources where green is strongest within the fire perimeter.

### Spatial Intervention Benefit Maps

**Four panels showing where each intervention helps:**

| Panel | Comparison | Question Answered |
|-------|------------|-------------------|
| **Top-left** | Fire Impact (fire_only - baseline) | Where did fire cause the most damage? |
| **Top-right** | Seeding Benefit (fire_seeding - fire_only) | Where does seeding help recovery? |
| **Bottom-left** | Removal Benefit (fire_removal - fire_only) | Where does invasive removal help? |
| **Bottom-right** | Combined Benefit (fire_both - fire_only) | Where does combined intervention help most? |

**Reading the maps:**
- Look for **green areas inside the fire perimeter** (black dashed line) - these are priority zones
- Areas outside the fire perimeter should be white (no intervention needed)
- The elevation contour (white dotted) helps identify areas that may recover naturally

![Spatial Difference Maps](analysis/figures/spatial/spatial_difference_maps.png)

### Figure 4: Fire Severity Map (Input)

**The fire pattern that drove all scenarios:**
- **Dark red (>0.7)**: High severity - most tree mortality, highest invasive potential
- **Orange (0.3-0.7)**: Moderate severity - partial mortality, competitive dynamics important  
- **Light/White (<0.3)**: Low severity - minor impacts, rapid natural recovery expected

This is the input disturbance pattern. The spatial difference maps above show how each intervention modifies outcomes relative to this fire pattern.

![Fire Severity Map](analysis/figures/spatial/fig4_fire_severity_map.png)

### Figure 12: Intervention Benefit Maps (R/ggplot2)

**Same concept as the Python figure above, rendered in ggplot2:**

Each panel shows additional trees gained (or lost) relative to fire-only at each location.
- **Seeding** panel: Where did planting seedlings help?
- **Removal** panel: Where did invasive removal help?
- **Both** panel: Where did combined intervention help?

Compare with the Python version above - same data, different visualization style.

![Intervention Benefit Maps](analysis/figures/spatial/fig12_intervention_benefit_maps.png)

### Figure 10: Tree Density at Year 50 - All Scenarios

**Side-by-side comparison of final tree density across all management scenarios:**

- **Baseline**: What the landscape looks like without fire (reference)
- **Fire Only**: Fire damage without intervention - note the "hole" in the center
- **Fire + Seeding**: Does planting fill in the fire damage?
- **Fire + Removal**: Does removing invasives allow natural recovery?
- **Fire + Both**: Combined intervention outcome

**Key observation**: Look for where the fire "hole" fills in across interventions.

![Tree Density Maps](analysis/figures/spatial/fig10_final_tree_density_maps.png)

### Figure 11: Invasive Cover at Year 50 - All Scenarios

**Invasive grass distribution across management scenarios:**

- **Dark green**: High invasive cover (blocking native establishment)
- **White/Light**: Low invasive cover (natives can establish)

**Key observations:**
- Compare the center of the fire scar across scenarios
- Removal scenarios should show less invasive cover in treated areas
- High invasive cover correlates with low tree density

![Invasive Cover Maps](analysis/figures/spatial/fig11_final_invasive_cover_maps.png)

---

## Conclusions and Interpretation

### What the Results Show

Compare the scenarios to answer management questions:

1. **Fire Impact** (`baseline` vs `fire_only`)  
   How much does fire reduce tree populations? How much does invasive cover increase?

2. **Seeding Effectiveness** (`fire_only` vs `fire_seeding`)  
   Does planting additional seedlings improve 50-year outcomes?

3. **Removal Effectiveness** (`fire_only` vs `fire_removal`)  
   Does removing invasive grasses improve native tree recovery?

4. **Synergy** (`fire_both` vs individual interventions)  
   Is the combination better than either intervention alone?

### Elevation-Driven Spatial Patterns

The key insight from including elevation is **spatial prioritization**:

| Zone | Fire + Elevation | Expected Outcome | Management Priority |
|------|------------------|------------------|---------------------|
| High elev (>60%), burned | Fire damage + favorable conditions | Natural recovery likely | Low |
| Medium elev (30-60%), burned | Fire damage + moderate conditions | Some recovery, seeding helps | Medium |
| Low elev (<30%), burned | Fire damage + unfavorable conditions | Invasive takeover without intervention | **High** |

**Interpretation guide for spatial maps:**
- **Negative correlation** between elevation and intervention benefit = interventions help most at low elevation (expected)
- **Positive correlation** between elevation and fire impact = fire hurt more at low elevation (where recovery is harder)

### Tuning the Elevation Effect

If the spatial patterns aren't clear enough, adjust parameters in `configs/params.jshc`:

| Parameter | Current | Effect | To increase pattern clarity |
|-----------|---------|--------|----------------------------|
| `invasiveElevationReduction` | 5% | Invasive growth reduction at high elevation | Increase to 7-8% |
| `seedlingElevationReduction` | 20% | Seedling mortality reduction at high elevation | Increase to 25-30% |
| `juvenileElevationReduction` | 6% | Juvenile mortality reduction at high elevation | Increase to 8-10% |

**Zone thresholds** are defined in `vegetation_model.josh`:
- High elevation: >= 60%
- Medium elevation: 30-60%
- Low elevation: < 30%

### Caveats

This is a **toy model** for demonstration purposes. It simplifies:

- No spatial seed dispersal between patches
- Single native tree species (vs. diverse community)
- No climate variability or drought effects
- Instantaneous interventions (vs. multi-year programs)
- Elevation effects are zone-based (discrete) rather than continuous gradients
- Elevation is synthetic (in reality, use real DEM data)

For real applications, the model would be calibrated with species-specific parameters and validated against monitoring data.

### Design Philosophy

This workflow embodies principles from the toolbox design:

| Principle | Implementation |
|-----------|----------------|
| **Partner-driven** | Addresses real questions: fire impact, intervention effectiveness, spatial prioritization |
| **Transparency** | All parameters explicit in Josh code; mechanisms inspectable |
| **Process-based** | Life stages, mortality, competition, elevation effects - not black-box ML |
| **Open tools** | Josh is open source; this notebook is fully reproducible |

### Next Steps

To extend this analysis:

- Run with multiple replicates (`--replicates 100`) to characterize stochastic variation
- Modify `configs/params.jshc` to test parameter sensitivity (especially elevation effects)
- Create new scenario configs to test alternative interventions
- Use real fire severity data from Sentinel-2 imagery
- Use real elevation data (DEM) instead of synthetic gradient